In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import util as u
import pipeline as p
import assess_clf_models as acm
%load_ext autoreload
%autoreload 2

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from skopt import BayesSearchCV
from joblib import dump, load

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=FutureWarning)
warnings.filterwarnings(action='ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

class BayesSearchCV(BayesSearchCV):
    def _run_search(self, x): raise BaseException('Use newer skopt')

import itertools

from sklearn.metrics import make_scorer, f1_score, confusion_matrix, precision_score, accuracy_score
from sklearn.metrics import precision_recall_curve, roc_auc_score, roc_curve, recall_score

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

### Load Data

In [2]:
X_train = u.open_pkl('Data/X_train_10yrs_02.pkl')
y_train = u.open_pkl('Data/y_train_10yrs_02.pkl')
X_test = u.open_pkl('Data/X_test_10yrs_02.pkl')
y_test = u.open_pkl('Data/y_test_10yrs_02.pkl')

In [4]:
# create sub-set if desired
if 1 == 1:
    size_X = len(X_train)
    rand_idx = np.random.randint(0, size_X - 1, size=int(size_X*0.1))
    mini_X = X_train.loc[rand_idx]
    mini_y = pd.Series([y_train.iloc[x] for x in rand_idx])
    mini_X.reset_index(inplace=True, drop=True)
    
    # pre-process & create mini test-train split
    mini_X_train, mini_X_test, mini_y_train, mini_y_test = train_test_split(
    mini_X, mini_y, test_size=0.3, random_state=19)

    proc_mini_X_train = pipe_cat_boost.fit_transform(mini_X_train)
    proc_mini_X_test = pipe_cat_boost.fit_transform(mini_X_test)

In [3]:
# Load pipeline info
pipe_cat_boost = u.open_pkl('Data/Models/cat_boost_preproc_01.pkl')
cb_cat_cols = u.open_pkl('Data/Models/cat_boost_categ_cols.pkl')

# pre-process inputs to expedite fitting
proc_X_train = pipe_cat_boost.fit_transform(X_train)
proc_X_test = pipe_cat_boost.fit_transform(X_test)

## Tune hyperparameters of CatBoost

Previously determined that model is fit by ~200 iterations (at depth 6)

In [4]:
scorer = make_scorer(f1_score, average='binary')

bay_opt = BayesSearchCV(CatBoostClassifier(random_seed=44, cat_features=cb_cat_cols, iterations=200), 
                        {'learning_rate': (1e-3, 0.5, 'log-uniform'),
                        'depth': (3,10), 
                        'l2_leaf_reg':(0.1, 10, 'log-uniform'),
                        'random_strength': (1, 20, 'uniform'),
                        'bagging_temperature': (0, 1, 'uniform'),
                                               }, 
                       n_iter=15, cv=5, scoring=scorer)

bay_opt.fit(proc_X_train, y_train)

0:	learn: 0.6545849	total: 186ms	remaining: 37s
1:	learn: 0.6167556	total: 290ms	remaining: 28.7s
2:	learn: 0.5822192	total: 386ms	remaining: 25.3s
3:	learn: 0.5497465	total: 497ms	remaining: 24.4s
4:	learn: 0.5185218	total: 602ms	remaining: 23.5s
5:	learn: 0.4890591	total: 705ms	remaining: 22.8s
6:	learn: 0.4616928	total: 820ms	remaining: 22.6s
7:	learn: 0.4350009	total: 935ms	remaining: 22.4s
8:	learn: 0.4099588	total: 1.04s	remaining: 22.2s
9:	learn: 0.3863974	total: 1.16s	remaining: 22s
10:	learn: 0.3643816	total: 1.27s	remaining: 21.8s
11:	learn: 0.3436849	total: 1.4s	remaining: 21.9s
12:	learn: 0.3238138	total: 1.49s	remaining: 21.5s
13:	learn: 0.3054752	total: 1.64s	remaining: 21.7s
14:	learn: 0.2883140	total: 1.79s	remaining: 22.1s
15:	learn: 0.2722385	total: 1.91s	remaining: 21.9s
16:	learn: 0.2568461	total: 2.01s	remaining: 21.6s
17:	learn: 0.2425828	total: 2.11s	remaining: 21.3s
18:	learn: 0.2289838	total: 2.23s	remaining: 21.3s
19:	learn: 0.2164208	total: 2.37s	remaining: 2

162:	learn: 0.0107294	total: 17.9s	remaining: 4.07s
163:	learn: 0.0107103	total: 18.1s	remaining: 3.96s
164:	learn: 0.0106771	total: 18.2s	remaining: 3.86s
165:	learn: 0.0106424	total: 18.3s	remaining: 3.75s
166:	learn: 0.0106040	total: 18.4s	remaining: 3.64s
167:	learn: 0.0105844	total: 18.6s	remaining: 3.53s
168:	learn: 0.0105560	total: 18.7s	remaining: 3.43s
169:	learn: 0.0105268	total: 18.8s	remaining: 3.32s
170:	learn: 0.0105042	total: 18.9s	remaining: 3.21s
171:	learn: 0.0104751	total: 19.1s	remaining: 3.1s
172:	learn: 0.0104474	total: 19.2s	remaining: 2.99s
173:	learn: 0.0104270	total: 19.3s	remaining: 2.88s
174:	learn: 0.0103996	total: 19.4s	remaining: 2.77s
175:	learn: 0.0103804	total: 19.5s	remaining: 2.66s
176:	learn: 0.0103531	total: 19.6s	remaining: 2.55s
177:	learn: 0.0103276	total: 19.7s	remaining: 2.44s
178:	learn: 0.0102984	total: 19.9s	remaining: 2.33s
179:	learn: 0.0102818	total: 20s	remaining: 2.22s
180:	learn: 0.0102666	total: 20.1s	remaining: 2.11s
181:	learn: 0.0

124:	learn: 0.0128624	total: 14.1s	remaining: 8.46s
125:	learn: 0.0127709	total: 14.2s	remaining: 8.34s
126:	learn: 0.0126892	total: 14.3s	remaining: 8.23s
127:	learn: 0.0126002	total: 14.4s	remaining: 8.11s
128:	learn: 0.0125311	total: 14.5s	remaining: 8s
129:	learn: 0.0124400	total: 14.6s	remaining: 7.88s
130:	learn: 0.0123756	total: 14.8s	remaining: 7.77s
131:	learn: 0.0123017	total: 14.8s	remaining: 7.65s
132:	learn: 0.0122082	total: 15s	remaining: 7.54s
133:	learn: 0.0121356	total: 15.1s	remaining: 7.43s
134:	learn: 0.0120643	total: 15.2s	remaining: 7.33s
135:	learn: 0.0119946	total: 15.3s	remaining: 7.22s
136:	learn: 0.0119346	total: 15.4s	remaining: 7.1s
137:	learn: 0.0118633	total: 15.5s	remaining: 6.98s
138:	learn: 0.0117962	total: 15.7s	remaining: 6.87s
139:	learn: 0.0117435	total: 15.8s	remaining: 6.76s
140:	learn: 0.0116875	total: 15.9s	remaining: 6.66s
141:	learn: 0.0116317	total: 16s	remaining: 6.54s
142:	learn: 0.0115789	total: 16.1s	remaining: 6.43s
143:	learn: 0.011529

86:	learn: 0.0194579	total: 10.3s	remaining: 13.4s
87:	learn: 0.0191361	total: 10.5s	remaining: 13.3s
88:	learn: 0.0188292	total: 10.6s	remaining: 13.2s
89:	learn: 0.0185260	total: 10.7s	remaining: 13.1s
90:	learn: 0.0182385	total: 10.9s	remaining: 13s
91:	learn: 0.0179650	total: 11s	remaining: 12.9s
92:	learn: 0.0177027	total: 11.1s	remaining: 12.7s
93:	learn: 0.0174475	total: 11.2s	remaining: 12.6s
94:	learn: 0.0172054	total: 11.3s	remaining: 12.5s
95:	learn: 0.0169707	total: 11.4s	remaining: 12.3s
96:	learn: 0.0167464	total: 11.5s	remaining: 12.2s
97:	learn: 0.0165233	total: 11.6s	remaining: 12s
98:	learn: 0.0163273	total: 11.7s	remaining: 12s
99:	learn: 0.0161240	total: 11.9s	remaining: 11.9s
100:	learn: 0.0159431	total: 12s	remaining: 11.8s
101:	learn: 0.0157619	total: 12.1s	remaining: 11.6s
102:	learn: 0.0155674	total: 12.2s	remaining: 11.5s
103:	learn: 0.0153967	total: 12.3s	remaining: 11.4s
104:	learn: 0.0152231	total: 12.4s	remaining: 11.2s
105:	learn: 0.0150573	total: 12.5s	r

48:	learn: 0.0520685	total: 5.62s	remaining: 17.3s
49:	learn: 0.0501178	total: 5.72s	remaining: 17.1s
50:	learn: 0.0482936	total: 5.82s	remaining: 17s
51:	learn: 0.0465192	total: 5.93s	remaining: 16.9s
52:	learn: 0.0448648	total: 6.04s	remaining: 16.8s
53:	learn: 0.0432865	total: 6.15s	remaining: 16.6s
54:	learn: 0.0418144	total: 6.26s	remaining: 16.5s
55:	learn: 0.0404099	total: 6.38s	remaining: 16.4s
56:	learn: 0.0390981	total: 6.49s	remaining: 16.3s
57:	learn: 0.0378179	total: 6.59s	remaining: 16.1s
58:	learn: 0.0366129	total: 6.7s	remaining: 16s
59:	learn: 0.0354884	total: 6.81s	remaining: 15.9s
60:	learn: 0.0344122	total: 6.92s	remaining: 15.8s
61:	learn: 0.0333929	total: 7.03s	remaining: 15.7s
62:	learn: 0.0324203	total: 7.14s	remaining: 15.5s
63:	learn: 0.0314990	total: 7.25s	remaining: 15.4s
64:	learn: 0.0306273	total: 7.36s	remaining: 15.3s
65:	learn: 0.0297877	total: 7.48s	remaining: 15.2s
66:	learn: 0.0289819	total: 7.59s	remaining: 15.1s
67:	learn: 0.0282243	total: 7.71s	re

...

175:	learn: 0.2552741	total: 39.1s	remaining: 5.33s
176:	learn: 0.2538542	total: 39.3s	remaining: 5.11s
177:	learn: 0.2524429	total: 39.5s	remaining: 4.88s
178:	learn: 0.2510398	total: 39.7s	remaining: 4.66s
179:	learn: 0.2496438	total: 39.9s	remaining: 4.43s
180:	learn: 0.2482580	total: 40.2s	remaining: 4.22s
181:	learn: 0.2468817	total: 40.5s	remaining: 4s
182:	learn: 0.2455139	total: 40.7s	remaining: 3.78s
183:	learn: 0.2441539	total: 40.9s	remaining: 3.56s
184:	learn: 0.2428009	total: 41.1s	remaining: 3.33s
185:	learn: 0.2414576	total: 41.7s	remaining: 3.14s
186:	learn: 0.2401213	total: 42.2s	remaining: 2.93s
187:	learn: 0.2387946	total: 42.5s	remaining: 2.71s
188:	learn: 0.2374760	total: 43s	remaining: 2.5s
189:	learn: 0.2361649	total: 43.3s	remaining: 2.28s
190:	learn: 0.2348628	total: 43.5s	remaining: 2.05s
191:	learn: 0.2335693	total: 43.7s	remaining: 1.82s
192:	learn: 0.2322834	total: 44.1s	remaining: 1.6s
193:	learn: 0.2310055	total: 44.4s	remaining: 1.37s
194:	learn: 0.22973

138:	learn: 0.3121633	total: 25.3s	remaining: 11.1s
139:	learn: 0.3103760	total: 25.5s	remaining: 10.9s
140:	learn: 0.3085985	total: 25.7s	remaining: 10.7s
141:	learn: 0.3068321	total: 25.9s	remaining: 10.6s
142:	learn: 0.3050765	total: 26.1s	remaining: 10.4s
143:	learn: 0.3033355	total: 26.2s	remaining: 10.2s
144:	learn: 0.3015910	total: 26.4s	remaining: 10s
145:	learn: 0.2998677	total: 26.6s	remaining: 9.84s
146:	learn: 0.2981496	total: 26.8s	remaining: 9.67s
147:	learn: 0.2964488	total: 27s	remaining: 9.49s
148:	learn: 0.2947471	total: 27.2s	remaining: 9.31s
149:	learn: 0.2930598	total: 27.4s	remaining: 9.15s
150:	learn: 0.2913839	total: 27.6s	remaining: 8.97s
151:	learn: 0.2897188	total: 27.8s	remaining: 8.79s
152:	learn: 0.2880603	total: 28s	remaining: 8.61s
153:	learn: 0.2864219	total: 28.2s	remaining: 8.43s
154:	learn: 0.2847927	total: 28.4s	remaining: 8.25s
155:	learn: 0.2831769	total: 28.6s	remaining: 8.07s
156:	learn: 0.2815690	total: 28.8s	remaining: 7.88s
157:	learn: 0.2799

100:	learn: 0.3945175	total: 19s	remaining: 18.7s
101:	learn: 0.3923154	total: 19.2s	remaining: 18.5s
102:	learn: 0.3901261	total: 19.4s	remaining: 18.3s
103:	learn: 0.3879500	total: 19.6s	remaining: 18.1s
104:	learn: 0.3857750	total: 19.8s	remaining: 17.9s
105:	learn: 0.3836131	total: 20.1s	remaining: 17.8s
106:	learn: 0.3814634	total: 20.2s	remaining: 17.6s
107:	learn: 0.3793374	total: 20.4s	remaining: 17.4s
108:	learn: 0.3772131	total: 20.6s	remaining: 17.2s
109:	learn: 0.3751118	total: 20.8s	remaining: 17s
110:	learn: 0.3730117	total: 21.1s	remaining: 16.9s
111:	learn: 0.3709344	total: 21.3s	remaining: 16.7s
112:	learn: 0.3688692	total: 21.4s	remaining: 16.5s
113:	learn: 0.3668080	total: 21.6s	remaining: 16.3s
114:	learn: 0.3647669	total: 21.8s	remaining: 16.1s
115:	learn: 0.3626953	total: 22.1s	remaining: 16s
116:	learn: 0.3606798	total: 22.3s	remaining: 15.8s
117:	learn: 0.3586749	total: 22.4s	remaining: 15.6s
118:	learn: 0.3566715	total: 22.6s	remaining: 15.4s
119:	learn: 0.3546

62:	learn: 0.0089490	total: 7.3s	remaining: 15.9s
63:	learn: 0.0089214	total: 7.42s	remaining: 15.8s
64:	learn: 0.0088971	total: 7.54s	remaining: 15.7s
65:	learn: 0.0088850	total: 7.65s	remaining: 15.5s
66:	learn: 0.0088629	total: 7.78s	remaining: 15.4s
67:	learn: 0.0088220	total: 7.9s	remaining: 15.3s
68:	learn: 0.0087851	total: 8.03s	remaining: 15.2s
69:	learn: 0.0087692	total: 8.13s	remaining: 15.1s
70:	learn: 0.0087320	total: 8.24s	remaining: 15s
71:	learn: 0.0087042	total: 8.36s	remaining: 14.9s
72:	learn: 0.0086749	total: 8.48s	remaining: 14.8s
73:	learn: 0.0086487	total: 8.59s	remaining: 14.6s
74:	learn: 0.0086080	total: 8.7s	remaining: 14.5s
75:	learn: 0.0085744	total: 8.82s	remaining: 14.4s
76:	learn: 0.0085626	total: 8.93s	remaining: 14.3s
77:	learn: 0.0085509	total: 9.05s	remaining: 14.1s
78:	learn: 0.0085403	total: 9.16s	remaining: 14s
79:	learn: 0.0085194	total: 9.27s	remaining: 13.9s
80:	learn: 0.0084954	total: 9.39s	remaining: 13.8s
81:	learn: 0.0084799	total: 9.51s	rema

BayesSearchCV(cv=5, error_score='raise',
       estimator=<catboost.core.CatBoostClassifier object at 0x1162f5860>,
       fit_params=None, iid=True, n_iter=15, n_jobs=1, n_points=1,
       optimizer_kwargs=None, pre_dispatch='2*n_jobs', random_state=None,
       refit=True, return_train_score=False,
       scoring=make_scorer(f1_score, average=binary),
       search_spaces={'learning_rate': (0.001, 0.5, 'log-uniform'), 'depth': (3, 10), 'l2_leaf_reg': (0.1, 10, 'log-uniform'), 'random_strength': (1, 20, 'uniform'), 'bagging_temperature': (0, 1, 'uniform')},
       verbose=0)

In [5]:
bay_opt.best_score_

0.575484084320476

In [6]:
bay_opt.best_params_

{'bagging_temperature': 0.5508606572877918,
 'depth': 7,
 'l2_leaf_reg': 0.8927857773878807,
 'learning_rate': 0.0843180263455367,
 'random_strength': 16.731958692351498}

In [9]:
cb_opt = bay_opt.best_estimator_

In [10]:
dump(cb_opt, 'Data/Models/cat_boost_opt_03.joblib')

['Data/Models/cat_boost_opt_03.joblib']

In [11]:
cb_opt.fit(proc_X_train, y_train)

0:	learn: 0.4200804	total: 135ms	remaining: 26.9s
1:	learn: 0.2527786	total: 267ms	remaining: 26.5s
2:	learn: 0.1565903	total: 419ms	remaining: 27.5s
3:	learn: 0.1013003	total: 545ms	remaining: 26.7s
4:	learn: 0.0688746	total: 691ms	remaining: 27s
5:	learn: 0.0493521	total: 827ms	remaining: 26.8s
6:	learn: 0.0369267	total: 993ms	remaining: 27.4s
7:	learn: 0.0290346	total: 1.14s	remaining: 27.4s
8:	learn: 0.0238718	total: 1.27s	remaining: 26.9s
9:	learn: 0.0203340	total: 1.38s	remaining: 26.3s
10:	learn: 0.0178857	total: 1.5s	remaining: 25.8s
11:	learn: 0.0160825	total: 1.62s	remaining: 25.5s
12:	learn: 0.0148057	total: 1.74s	remaining: 25s
13:	learn: 0.0138471	total: 1.94s	remaining: 25.8s
14:	learn: 0.0131650	total: 2.05s	remaining: 25.3s
15:	learn: 0.0126302	total: 2.17s	remaining: 25s
16:	learn: 0.0121903	total: 2.32s	remaining: 24.9s
17:	learn: 0.0118304	total: 2.45s	remaining: 24.8s
18:	learn: 0.0115720	total: 2.58s	remaining: 24.6s
19:	learn: 0.0113557	total: 2.69s	remaining: 24.

161:	learn: 0.0070727	total: 21.7s	remaining: 5.09s
162:	learn: 0.0070594	total: 21.8s	remaining: 4.95s
163:	learn: 0.0070396	total: 21.9s	remaining: 4.82s
164:	learn: 0.0070270	total: 22.1s	remaining: 4.7s
165:	learn: 0.0070105	total: 22.3s	remaining: 4.56s
166:	learn: 0.0069922	total: 22.4s	remaining: 4.42s
167:	learn: 0.0069797	total: 22.5s	remaining: 4.29s
168:	learn: 0.0069726	total: 22.6s	remaining: 4.15s
169:	learn: 0.0069570	total: 22.8s	remaining: 4.02s
170:	learn: 0.0069450	total: 22.9s	remaining: 3.89s
171:	learn: 0.0069367	total: 23.1s	remaining: 3.75s
172:	learn: 0.0069262	total: 23.2s	remaining: 3.62s
173:	learn: 0.0069119	total: 23.3s	remaining: 3.48s
174:	learn: 0.0069042	total: 23.4s	remaining: 3.35s
175:	learn: 0.0068891	total: 23.6s	remaining: 3.21s
176:	learn: 0.0068735	total: 23.7s	remaining: 3.08s
177:	learn: 0.0068592	total: 23.8s	remaining: 2.94s
178:	learn: 0.0068408	total: 24s	remaining: 2.82s
179:	learn: 0.0068342	total: 24.2s	remaining: 2.69s
180:	learn: 0.0

In [13]:
y_pred = cb_opt.predict(proc_X_test)
y_proba = cb_opt.predict_proba(proc_X_test)[:,1]

u.pkl_this('Data/Models/y_pred_cb_10yrs_02.pkl', y_pred) 
u.pkl_this('Data/Models/y_proba_cb_10yrs_02.pkl', y_proba)

If time: investigate impact of class_weights

In [15]:
conf_mat = confusion_matrix(y_test, y_pred)

In [16]:
precs = precision_score(y_test,y_pred, average=None)
recalls = recall_score(y_test, y_pred, average=None)
f1s = f1_score(y_test, y_pred, average=None)
aucs = roc_auc_score(y_test,y_proba)
accs = accuracy_score(y_test, y_pred)

twos = [precs, recalls, f1s]
data1 = [[s[j]] for j in range(2) for s in twos]
data1_new = np.mean(data1, axis=1).reshape(1,6)
df1 = pd.DataFrame(data1_new,
         columns=['Precision-0', 'Recall-0 (Specificty)','F1score-0','Precision-1',
         'Recall-1 (Sensitivity)','F1score-1']).mean()
data2 = conf_mat.flatten().reshape(-1,4)
df2 = pd.DataFrame(data2, columns=['TN','FP','FN','TP']).mean()
data3 = np.array([aucs, accs]).reshape(-1, 2)
df3 = pd.DataFrame(data3, columns=['AUC','Accuracy']).mean()

df = df1.append(df2.append(df3))
print(df)

Precision-0                    0.997687
Recall-0 (Specificty)          0.999563
F1score-0                      0.998625
Precision-1                    0.821326
Recall-1 (Sensitivity)         0.464169
F1score-1                      0.593132
TN                        141937.000000
FP                            62.000000
FN                           329.000000
TP                           285.000000
AUC                            0.984186
Accuracy                       0.997258
dtype: float64


In [17]:
fis = cb_opt.feature_importances_

In [19]:
fis

array([ 8.05786486,  0.80305176,  1.34830229,  2.05608784,  2.71634458,
        0.15103509,  0.02379156,  0.52071322,  2.42793443,  1.69353873,
       10.34457446,  3.08421045,  4.42156242,  4.66034727,  5.96193467,
        8.1398323 ,  2.43220683,  1.2498866 ,  3.7869942 ,  4.31738516,
        0.0218393 ,  0.35341947,  3.05812122, 12.39943711,  1.7340931 ,
        3.89790895,  1.61148463,  0.96623593,  3.04225786,  4.02427019,
        0.49126304,  0.        ,  0.20207048])

In [20]:
u.pkl_this('Data/Models/cat_boost_opt_03_fis.pkl', fis)